# Migration Flow in the US from 2010 to 2019

The dataset for this project originates from the US Census Bureau. The dataset contains the State-to-State migration flows for 10 years from 2010 to 2019. The data has some interesting information about the population reside in each state, number of people resided in different state one year ago, and number of people was abroad one year ago.

## First clance at the dataset
The dataset has 10 Excel files which format in multiple headers and comments. The number of rows and columns in each data file is the same. 

## Exploring The Data
Here are some steps for exploring the data.
- Need to clean the dataset by removing all the titles and comments.
- Eliminate the MOE columns in all files.
- Set row 6 to be header and rename the column headers.
- Merge all the data files and transform to long format.

Let’s start with importing the necessary libaries, reading in the data, and checking out the dataset.

In [1]:
# Import libaries
import pandas as pd
import numpy as np
import math
import glob
import re
import os

# Import all migration data from 2010 to 2019
path = r'raw_data' # use your path
all_files = glob.glob(path + '/*.xls')

li = []

for filename in all_files:
    df = pd.read_excel(filename,na_values=['(NA)'], header = 6).fillna(0)
    
    # Remove all the MOE columns and the Footnotes which are from row 70 to 76 for all the data files
    df = df.loc[:70,(df != 'MOE').all()]

    # Adjust the column names
    df.rename(columns=
              {df.columns[0]: 'destination_state',
               df.columns[1]: 'population',
               df.columns[2]: 'same_house',
               df.columns[3]: 'same_state',
               df.columns[4]: 'from_different_state_Total',
               df.columns[-4]: 'abroad_Total',
               df.columns[-3]: 'abroad_PuertoRico',
               df.columns[-2]: 'abroad_USIslandArea',
               df.columns[-1]: 'abroad_ForeignCountry'
              }, inplace = True)

    #for i in range(4,56):
    #    df.rename(columns = {df.columns[i]:'from_state_' + df.columns[i]}, inplace = True)

    # Remove NA rows and duplicate columns
    df = df[df['destination_state'] != 0]
    df = df[df.columns.drop(list(df.filter(regex='Unnamed:')))]
    df = df.drop([2])
    df = df.drop([37])

    # Rename the first element and set the first column as index
    # df.values[0][0] = 'United States'
    
    # Change all columns to integer
    for col in df.columns[1:]:
        df[col] = df[col].astype('int64')
        
    # Create a column to store the year based on the Excel file name
    df['year'] = re.findall('\d+', os.path.basename(filename))[0]
    
    # Reset index
    df.reset_index(drop = True, inplace = True)
        
    li.append(df)

# Check out the first 5 rows of one of the file
li[1].head()

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,4745278,4024442,588293,117726,0,1771,1677,1642,3389,...,4935,2621,65,417,9,14817,569,0,14248,2011
1,Alaska,711962,571857,100280,35084,93,0,2467,190,3098,...,1488,4548,89,23,246,4741,1044,0,3697,2011
2,Arizona,6402301,5107496,1028366,222877,833,5001,0,1066,49635,...,2233,13940,70,6473,2510,43562,871,122,42569,2011
3,Arkansas,2906632,2421746,405831,69845,691,560,439,0,4077,...,1245,1477,24,687,252,9210,529,46,8635,2011
4,California,37222678,31213310,5271168,468428,2087,7358,35650,2648,0,...,15753,36481,832,5668,2047,269772,1344,2817,265611,2011


Concatenate all dataframes and transform to long format

In [2]:
migration_df = pd.concat(li)
migration_df

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,4729509,3987155,620465,108723,0,3013,676,1481,3827,...,2490,1171,41,1155,27,13166,228,181,12757,2010
1,Alaska,702974,565031,95878,36326,477,0,1354,47,3906,...,714,2421,0,158,81,5739,19,1374,4346,2010
2,Arizona,6332786,5069002,1001991,222725,416,3109,0,689,47164,...,3413,12645,595,5556,593,39068,599,223,38246,2010
3,Arkansas,2888304,2387806,412997,79127,1405,934,777,0,4457,...,494,264,0,821,443,8374,87,0,8287,2010
4,California,36907897,30790221,5413287,444749,3364,9579,33854,4172,0,...,14232,30544,1446,6031,1336,259640,1223,5123,253294,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,Washington,7527366,6253469,977928,231956,1060,4255,8263,83,46791,...,6656,0,275,1456,626,64013,318,1465,62230,2019
48,West Virginia,1773280,1563611,164739,39548,245,192,356,120,940,...,6008,129,0,41,0,5382,606,0,4776,2019
49,Wisconsin,5760481,5001140,634732,107973,678,1070,1917,290,6886,...,1878,1682,572,0,1417,16636,513,0,16123,2019
50,Wyoming,572884,473128,68127,30247,1227,54,757,298,3211,...,415,885,308,27,0,1382,0,0,1382,2019


In [3]:
id_var = ['destination_state','year', 'population','same_house', 'same_state','from_different_state_Total','abroad_Total']
id_values = [col for col in migration_df.columns if col not in id_var]
df_long = pd.melt( migration_df,id_var , id_values, 'from', 'number_of_people' )
df_long

,destination_state,year,population,same_house,same_state,from_different_state_Total,abroad_Total,from,number_of_people
0,Alabama,2010,4729509,3987155,620465,108723,13166,Alabama,0
1,Alaska,2010,702974,565031,95878,36326,5739,Alabama,477
2,Arizona,2010,6332786,5069002,1001991,222725,39068,Alabama,416
3,Arkansas,2010,2888304,2387806,412997,79127,8374,Alabama,1405
4,California,2010,36907897,30790221,5413287,444749,259640,Alabama,3364
...,...,...,...,...,...,...,...,...,...
28075,Washington,2019,7527366,6253469,977928,231956,64013,abroad_ForeignCountry,62230
28076,West Virginia,2019,1773280,1563611,164739,39548,5382,abroad_ForeignCountry,4776
28077,Wisconsin,2019,5760481,5001140,634732,107973,16636,abroad_ForeignCountry,16123
28078,Wyoming,2019,572884,473128,68127,30247,1382,abroad_ForeignCountry,1382


Export to CSV file and use this data to make [Tableau Dashboard](https://public.tableau.com/profile/finnegan.nguyen#!/vizhome/MigrationFlow10years2010-2019/Dashboard1)

In [4]:
# df_long.to_csv('migration_10years.csv')